In [1]:
import json 
import redis
import cherrypy

seriesName = "emergencySiren"

In [2]:
#open redis connection
REDIS_HOST = "redis-17637.c114.us-east-1-4.ec2.cloud.redislabs.com"
REDIS_PORT = 17637
REDIS_USER = "default"
REDIS_PASSWORD = "fFh38GR6BxHI0xVoGbsUKAzPZlCBXLAA"

redis_client = redis.Redis(host = REDIS_HOST, port = REDIS_PORT, password = REDIS_PASSWORD, username= REDIS_USER)
print("Is connected:", redis_client.ping())


Is connected: True


In [3]:
class DeleteSensorHistory:
    """Either deletes the history a single sensor or all sensors history depending on the path"""
    exposed = True

    def DELETE(self, *path, **query):
        if len(path) == 0:
            sensorList = [name.decode().split(":")[0] for name in redis_client.keys(f"*:{seriesName}")]
            for sensor in sensorList:
                redis_client.delete(f"{sensor}:{seriesName}")
        else:
                redis_client.delete(f"{path[0]}:{seriesName}")

        
class ListSensors:
    """List all sensor names"""
    exposed = True

    def GET(self, *path, **query):
        sensorList = [name.decode().split(":")[0] for name in redis_client.keys(f"*:{seriesName}")]
        output_dict = {"sensor" : sensorList}
        return json.dumps(output_dict)


class SensorHistory:
    """If no path and no query then returns all the sensors histories
       If only path and no query then returns the specific senso's history
       If only query and no path then return all sensors' history in the specific range
       If both path and query returns the spefic sensor's history in the specific range"""

    exposed = True

    def GET(self, *path, **query):
        output_dict = {}

        #Check for query parameters 
        if query.get("from") is None:
            start = "-"
        else:
            start = query.get("from")
        if query.get("to") is None:
            end = "+"
        else:
            end = query.get("to")

        #If any sensor is provided
        if len(path) == 0:
            sensorList = [name.decode().split(":")[0] for name in redis_client.keys(f"*:{seriesName}")]
            for sensor in sensorList:
                emergencySirens = redis_client.ts().range(f"{sensor}:{seriesName}", start, end)
                output_dict[sensor] = {
                    "timestamps" : [ts for ts, bl in emergencySirens],
                    "emergencySirens" : [bl for ts, bl in emergencySirens]
                }

        #If a sensor is provided
        else:
            sensor = path[0]
            if len(redis_client.keys(f"{sensor}:*"))==0:
                raise cherrypy.HTTPError(404, 'Not Found: invalid sensor name.')
            else: 
                emergencySirens = redis_client.ts().range(f"{sensor}:{seriesName}", start, end)
                output_dict[sensor] = {
                    "timestamps" : [ts for ts, bl in emergencySirens],
                    "emergencySirens" : [bl for ts, bl in emergencySirens]
                }

        return json.dumps(output_dict)


In [4]:
if __name__== '__main__':
    conf = {
        '/': {'request.dispatch': cherrypy.dispatch.MethodDispatcher()}
    }

    #create link between the class and the endpoint
    cherrypy.tree.mount(DeleteSensorHistory(), '/delete', conf)
    cherrypy.tree.mount(ListSensors(), '/list', conf)
    cherrypy.tree.mount(SensorHistory(), '/sensor', conf)
  
    #specify the IP address of the host and host
    cherrypy.config.update({'server.socket_host': '0.0.0.0'})
    cherrypy.config.update({'server.socket_port': 8080}) 

    #start and keep it online
    cherrypy.engine.start()
    cherrypy.engine.block()

[19/Apr/2023:09:47:53] ENGINE Bus STARTING
[19/Apr/2023:09:47:53] ENGINE Started monitor thread 'Autoreloader'.
[19/Apr/2023:09:47:54] ENGINE Serving on http://0.0.0.0:8080
[19/Apr/2023:09:47:54] ENGINE Bus STARTED
172.3.150.40 - - [19/Apr/2023:09:48:32] "GET /sensor?from=1681897710958&to=1681897711958 HTTP/1.1" 200 57 "" "python-requests/2.28.1"
172.3.150.40 - - [19/Apr/2023:09:48:38] "GET /sensor?from=1681897717538&to=1681897718538 HTTP/1.1" 200 57 "" "python-requests/2.28.1"
172.3.22.214 - - [19/Apr/2023:09:48:40] "GET /sensor?from=1681897719901&to=1681897720901 HTTP/1.1" 200 57 "" "python-requests/2.28.1"
172.3.150.40 - - [19/Apr/2023:09:48:43] "GET /sensor?from=1681897722518&to=1681897723518 HTTP/1.1" 200 57 "" "python-requests/2.28.1"
172.3.150.40 - - [19/Apr/2023:09:48:52] "GET /sensor?from=1681897730064&to=1681897732064 HTTP/1.1" 200 73 "" "python-requests/2.28.1"
172.3.18.60 - - [19/Apr/2023:09:48:57] "GET /sensor?from=1681897735337&to=1681897737337 HTTP/1.1" 200 73 "" "python

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=957cfe07-d816-4770-9cc0-d7c1b4767886' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>